In [1]:
import pandas as pd

# Function to downcast numeric columns
def downcast(df):
    for col in df.select_dtypes(include=['int']).columns:
        df.loc[:, col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float']).columns:
        df.loc[:, col] = pd.to_numeric(df[col], downcast='float')
    return df

# Load the datasets with only the required columns
df1 = pd.read_csv('C:/Users/Guest01/Documents/dataset_anime/archive/final_animedataset.csv', usecols=['user_id', 'anime_id', 'my_score', 'score', 'scored_by', 'gender'])
df2 = pd.read_csv('C:/Users/Guest01/Documents/dataset_anime/archive/users-score-2023.csv', usecols=['user_id', 'anime_id', 'Anime Title', 'rating'])

# Merge the datasets on user_id and anime_id
merged_df = pd.merge(df2, df1, on=['user_id', 'anime_id'])

# Select the necessary columns for the final DataFrame
final_df = merged_df[['user_id', 'anime_id', 'my_score', 'score', 'scored_by', 'gender', 'Anime Title', 'rating']]

# Downcast numeric columns
final_df = downcast(final_df)

# Display the first few rows to check the data
print(final_df.head())


   user_id  anime_id  my_score  score  scored_by gender  \
0        1        21         9   8.54     423868   Male   
1        1        48         7   7.09      61485   Male   
2        1       320         5   6.66      18934   Male   
3        1        49         8   7.38      20930   Male   
4        1       304         8   7.63      18571   Male   

              Anime Title  rating  
0               One Piece       9  
1             .hack//Sign       7  
2                  A Kite       5  
3        Aa! Megami-sama!       8  
4  Aa! Megami-sama! Movie       8  


In [2]:
# Handle missing values
final_df['my_score'].fillna(final_df['my_score'].median(), inplace=True)
final_df['score'].fillna(final_df['score'].median(), inplace=True)
final_df['scored_by'].fillna(final_df['scored_by'].median(), inplace=True)

# Frequency encoding for categorical columns
categorical_columns = ['gender', 'Anime Title']

for col in categorical_columns:
    freq = final_df[col].value_counts()
    final_df[col] = final_df[col].map(freq)

# Drop any remaining rows with missing values if necessary
final_df = final_df.dropna()

# Display the first few rows after preprocessing
print(final_df.head())


C:\Users\Guest01\AppData\Local\Temp\ipykernel_9900\1031953128.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['my_score'].fillna(final_df['my_score'].median(), inplace=True)
C:\Users\Guest01\AppData\Local\Temp\ipykernel_9900\1031953128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['my_score'].fillna(final_df['my_sco

   user_id  anime_id  my_score  score  scored_by   gender  Anime Title  rating
0        1        21         9   8.54     423868  4175028        10462       9
1        1        48         7   7.09      61485  4175028         5341       7
2        1       320         5   6.66      18934  4175028         1892       5
3        1        49         8   7.38      20930  4175028         2448       8
4        1       304         8   7.63      18571  4175028         2395       8


In [3]:
# Define features (X) and target (y)
X = final_df.drop(columns=['rating'])
y = final_df['rating']

# Display the shapes of X and y to confirm
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (6021419, 7), y shape: (6021419,)


In [4]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil

# Function to calculate and return metrics
def calculate_metrics(X_train, X_test, y_train, y_test):
    # Define the model
    dt = DecisionTreeRegressor(random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'max_depth': [None, 10, 20, 30, 40, 50, 60, 80, 100, 150, 200],
        'min_samples_split': [2, 5, 10, 20, 30, 40, 50],
        'min_samples_leaf': [1, 2, 4, 8, 10],
        'max_features': ['sqrt', 'log2'], 
        'max_leaf_nodes': [50, 100, 200, 300, 400, 500],
    }

    random_search = RandomizedSearchCV(
        dt, 
        param_distributions, 
        n_iter=20, 
        cv=5, 
        scoring='neg_mean_squared_error', 
        random_state=42, 
        n_jobs=-1
    )

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    # Make predictions
    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [6]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total       
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Metrics for sample size 1.0:
RMSE: 0.5058724705994067
MAPE: 0.046616498040820915
R2: 0.9079193886867182
MSE: 0.2559069565103475
nRMSE: 0.05620805228882296
Execution Time (Raw): 168.43912935256958
Normalized Time (s/MB): 0.4587738257861451
Average CPU Usage: 54.85
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 0.5082165559785111
MAPE: 0.043813541846095504
R2: 0.9070018645197266
MSE: 0.2582840677706591
nRMSE: 0.05646850621983457
Execution Time (Raw): 76.5824863910675
Normalized Time (s/MB): 0.4167548099239249
Average CPU Usage: 60.85
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 0.5729982594780545
MAPE: 0.057087550521590966
R2: 0.8817821029755977
MSE: 0.3283270053648799
nRMSE: 0.06366647327533939
Execution Time (Raw): 34.784424781799316
Normalized Time (s/MB): 0.37858737255489405
Average CPU Usage: 55.5
Sample Size: 0.25
--------------------------------------------------
Metr

In [7]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 4 minutes and 58.18 seconds
Total Normalized Execution Time for Entire Process: 0.42956210 seconds per MB
Total Average CPU Usage for Entire Process: 49.07%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.505872,0.046616,0.907919,0.255907,0.056208,168.439129,0.458774,54.85,1.000
1,0.508217,0.043814,0.907002,0.258284,0.056469,76.582486,0.416755,60.85,0.500
2,0.572998,0.057088,0.881782,0.328327,0.063666,34.784425,0.378587,55.50,0.250
3,0.559609,0.059684,0.887243,0.313162,0.062179,15.858000,0.345191,55.15,0.125
4,2.084203,0.302556,0.003692,4.343903,0.260525,0.134485,27.542529,42.00,100.000
5,0.810268,0.077329,0.759015,0.656535,0.090030,0.149424,3.060210,35.85,1000.000
6,0.662567,0.057669,0.851951,0.438995,0.073619,0.266858,0.546525,45.00,10000.000
7,0.549436,0.052856,0.891765,0.301880,0.061048,1.967390,0.402921,43.35,100000.000
